In [1]:
import pandas as pd
import geopandas as gpd
import movingpandas as mpd
import numpy as np
from datetime import timedelta, datetime
import folium
import time
import warnings
import pickle
import sys

warnings.filterwarnings('ignore')

print("Geopandas has version {}".format(gpd.__version__))
print("Movingpandas has version {}".format(mpd.__version__))

Geopandas has version 0.13.2
Movingpandas has version 0.17.1


In [2]:
# add paths for modules
sys.path.append('../models')
# import modules
import visualize
from maritime_traffic_network import MaritimeTrafficNetwork

In [3]:
# load network from pickle
model = '202204_waypoints_DP10_HDBSCAN12_tromso_full_UTM'
model_path = '../../models/networks/best_networks/' + model + '.obj'
fileObj = open(model_path, 'rb')
network = pickle.load(fileObj)
fileObj.close()
network.hyperparameters

{'Data': '../../data/processed/202204_points_tromso_cleaned_meta_full_dualSplit_2.parquet',
 'DP_tolerance': 10,
 'clustering_method': 'HDBSCAN',
 'clustering_metric': 'mahalanobis',
 'clustering_min_samples': 12,
 'clustering_min_cluster_size': 12,
 'clustering_eps': 0,
 'clustering_metric_V': array([[1.e+00, 0.e+00, 0.e+00, 0.e+00, 0.e+00],
        [0.e+00, 1.e+00, 0.e+00, 0.e+00, 0.e+00],
        [0.e+00, 0.e+00, 1.e-02, 0.e+00, 0.e+00],
        [0.e+00, 0.e+00, 0.e+00, 1.e-02, 0.e+00],
        [0.e+00, 0.e+00, 0.e+00, 0.e+00, 5.e-04]]),
 'graph_generation_max_distance': 10,
 'graph_generation_max_angle': 45}

In [4]:
# merge stop points
network.merge_stop_points(max_speed=2)

In [5]:
# prune
network.prune_graph(1)

Pruning...
------------------------
Pruned Graph:
Number of nodes: 565 (1 isolated)
Number of edges: 2181
------------------------


In [6]:
# plot network and basemap
network.prune_graph(100)
map = network.map_graph(pruned=True)  #tromso: center=[69, 19]
network.prune_graph(1)
# add some trajectories
all_trajectories = network.trajectories
selection = [83] # [56, 23, 83, 99, 197, 497, 372]
mmsis = network.gdf.mmsi.unique()[selection]
trajectories = all_trajectories.filter('mmsi', mmsis.tolist())
for mmsi in mmsis:
    trajectory = all_trajectories.get_trajectory(mmsi)
    start = time.time()
    path_df2, evaluation_results2 = network.trajectory_to_path2(trajectory, verbose=True)
    end = time.time()  # end timer
    print(f'New time: {(end-start):.2f} seconds')
    start = time.time()
    path_df, evaluation_results = network.trajectory_to_path(trajectory, verbose=False)
    end = time.time()  # end timer
    print(f'Old time: {(end-start):.2f} seconds')
    #print(mmsi, evaluation_results['message'])
    #print(mmsi, evaluation_results2['message'])
    try:
        map = trajectory.to_line_gdf()[['geometry', 'mmsi', 'skipsgruppe']].explore(m=map, style_kwds={'weight':3, 'color':'black', 'opacity':1}, 
                                                                      name=mmsi+' trajectory')
        map = path_df.explore(m=map, name=mmsi+' edge_sequence ', style_kwds={'weight':3, 'color':'yellow', 'opacity':1})
        map = path_df2.explore(m=map, name=mmsi+' greedy ', style_kwds={'weight':3, 'color':'orange', 'opacity':1})
    except:
        print(mmsi+': no path found')

folium.LayerControl().add_to(map)
#map

Pruning...
------------------------
Pruned Graph:
Number of nodes: 565 (1 isolated)
Number of edges: 1453
------------------------
Pruning...
------------------------
Pruned Graph:
Number of nodes: 565 (1 isolated)
Number of edges: 2181
------------------------
231766000_1_2022-04-11 01:28:23
Intersections found: [289, 7, 482, 481, 479, 487, 486, 485, 480, 28, 475, 470, 465, 31]
Executing try statement
From: 289  To: 7
Iterating through edge sequences
Edge sequence: [289, 7]
Point indices: 3 17
   Mean distance: 58.073361516439235
Edge sequence: [289, 428, 7]
Point indices: 3 17
   Mean distance: 119.23468688653813
----------------------
From: 7  To: 482
Iterating through edge sequences
Edge sequence: [7, 482]
Point indices: 17 22
   Mean distance: 21.88898991259057
----------------------
From: 482  To: 481
Iterating through edge sequences
Edge sequence: [482, 481]
Point indices: 22 29
   Mean distance: 27.2311352847003
----------------------
From: 481  To: 479
Iterating through edge s

NameError: name 'length_ratio' is not defined

In [ ]:
map.save('../../reports/maps/'+model+'.html')

In [ ]:
evaluation_results

In [ ]:
network.plot_graph_canvas()